In [ ]:
tiles = dl.raster.dltiles_from_shape(
    resolution= 60.0, 
    tilesize=2048, 
    pad=16, 
    shape=shape)
pprint(tiles['features'][0])
pprint("Total number of tiles for Zimbabwe: " + str(len(tiles['features'])))